## メモ

ファイル名の変更などは diff --git a/\~. b/\~. 
<br>上記 a, bで示される

## データ分析用コード

In [139]:
import pandas as pd
import os, re
from IPython.display import display
from tqdm import tqdm

In [140]:
saikyo = pd.read_csv('./saikyo-data.csv')
saikyo.head(1)

,id,project,path,revision,url,link,start,end,isLicense,isAutoGenerated,isIncludedKeyword,satdKeyword,isIncludedOurKeyword,ourKeyword,isSelected,comment,below,class
0,1,31z4/storm-docker,31z4/storm-docker/1.1.3/Dockerfile\n31z4/storm...,664a7f4730f58be00fd7fe0526d2c13180107c6f,https://github.com/31z4/storm-docker/blob/664a...,https://github.com/31z4/storm-docker/blob/664a...,7\n7\n7,7\n7\n7,0,0,0,NaN,0,NaN,0.0,# Add a user with an explicit UID/GID and crea...,RUN set -eux; \\nRUN set -eux; \\nRUN set -eux; \,NaN


## 分析作業② 以後ここから実行
①では Dockerfileを含むコミットを取得しているだけなので
<br>コメントが付与された時のファイル名を保存しておき、そのファイルが含まれるコミットかどうかの判断も追加してみる

In [145]:
results = os.listdir('./preprocessed_rename')
outputs = os.listdir('./onehot-encoding')


def colname_modify(col):
    if col[:2] == "a/":
        col = col.split(' ')[0][2:]
    return col
    

for i, result in enumerate(results):
    
    ## ファイル拡張子が違うものは除外
    if result[-4:] != '.csv':
        continue

    ## 実行済みのものは ignore
    if result in outputs:
        continue
        
    df = pd.read_csv(f'./preprocessed_rename/{result}', index_col=0)
    print(f'\n============= {i}/{len(results)}')
    print(result)

    ## コメントが付与された行取得
    last_check = df.copy(deep=True)
    last_check = last_check[ last_check["diff"] != str(0)]
    
    ## コメントが付与されたファイル名取得
    files = last_check["diff_files"].values
    files = re.findall(r'\'([^\s]*)\s[a/|b/][^\']*\'', str(files))
    files = list(set(files)) ##重複削除

    ## 日付の形式を変更
    df = df[df["date"] != "[]"]
    df["date"] = pd.to_datetime(df["date"].apply(lambda dt: re.findall(r'\D{3}\s\D{3}\s\d*\s\d{2}:\d{2}:\d{2}\s\d{4}\s[-|+]\d{4}', dt)[0]))

    ## ワンホットエンコーディング的なもの
    count = {"commitid": [], "date": [], "diff": []}
    for file in files:
        count[str(file)] = []
    
    for index, row in df.iterrows():
        ## ======= 通常用  ==========
        count["commitid"].append(row["commitid"])
        count["date"].append(row["date"])
        count["diff"].append(0)
        
        for file in files:
            if file in row["file"]:
                count[str(file)].append(1)          
            else:
                count[str(file)].append(0)
        
        ## ======= Diff用  ==========
        if row["diff"] != "0":
            count["commitid"].append(row["commitid"])
            count["date"].append(row["date"])
            count["diff"].append(1)
            
            for file in files:
                if file in row["diff_files"]:
                    count[str(file)].append(1)          
                else:
                    count[str(file)].append(0)
        
    df = pd.DataFrame.from_dict(count)
    df.columns = list(map(lambda col: colname_modify(col),  df.columns.tolist()))
    
    try: ## Diffは存在していたけどSATD追加ではなかったときに [] が表示されてしまう
        df.drop(columns="[]", inplace=True)
    except KeyError:
        pass

    df.to_csv(f'./onehot-encoding/{result}')


============= 0/409
2141_ghost_nondebt.csv

============= 1/409
1612_dockerfiles.csv

============= 2/409
508_php_nondebt.csv

============= 3/409
2387_AIforEarth-API-Development_nondebt.csv

============= 4/409
1507_docker-splunk_nondebt.csv

============= 5/409
1588_dockerfiles.csv

============= 6/409
1877_batch-shipyard_nondebt.csv

============= 7/409
269_docker.csv

============= 8/409
1424_rocker_nondebt.csv

============= 9/409
1159_docker-nuxeo_nondebt.csv

============= 10/409
737_compose.csv

============= 11/409
1733_mediawiki-docker.csv

============= 12/409
2433_cntk_nondebt.csv

============= 13/409
463_openjdk.csv

============= 14/409
2915_PowerShell-Docker_nondebt.csv

============= 15/409
2032_bosh-deployment-resource_nondebt.csv

============= 16/409
702_tomcat_nondebt.csv

============= 17/409
2790_newman_nondebt.csv

============= 18/409
2460_cntk.csv

============= 19/409
472_openjdk_nondebt.csv

============= 20/409
944_mq-container_nondebt.csv

============= 2


============= 168/409
1191_ci.docker_nondebt.csv

============= 169/409
309_elasticsearch_nondebt.csv

============= 170/409
207_docker_nondebt.csv

============= 171/409
1995_MachineLearningNotebooks_nondebt.csv

============= 172/409
1526_docker-bash_nondebt.csv

============= 173/409
172_convertigo_nondebt.csv

============= 174/409
1543_dockerfiles.csv

============= 175/409
2668_vsts-agent-docker_nondebt.csv

============= 176/409
1969_batch-shipyard_nondebt.csv

============= 177/409
941_mq-container_nondebt.csv

============= 178/409
1441_rocker_nondebt.csv

============= 179/409
1015_gateway.docker_nondebt.csv

============= 180/409
2064_cp-docker-images_nondebt.csv

============= 181/409
1167_docker-nuxeo_nondebt.csv

============= 182/409
2948_PowerShell-Docker_nondebt.csv

============= 183/409
2261_Known-docker_nondebt.csv

============= 184/409
1671_dockerfiles_nondebt.csv

============= 185/409
1091_dockerfiles_nondebt.csv

============= 186/409
97_docker-jetty_nondebt.c


============= 330/409
1763_spark_nondebt.csv

============= 331/409
892_vault_nondebt.csv

============= 332/409
789_docker-ce_nondebt.csv

============= 333/409
3015_docker-selenium_nondebt.csv

============= 334/409
1338_docker_images.csv

============= 335/409
1601_dockerfiles_nondebt.csv

============= 336/409
292_docs.csv

============= 337/409
1101_docker_nondebt.csv

============= 338/409
293_docs.csv

============= 339/409
1157_docker-nuxeo_nondebt.csv

============= 340/409
2205_samba_nondebt.csv

============= 341/409
1875_batch-shipyard_nondebt.csv

============= 342/409
1903_batch-shipyard_nondebt.csv

============= 343/409
1221_orientdb-docker_nondebt.csv

============= 344/409
2505_iis-docker_nondebt.csv

============= 345/409
2281_docker_nondebt.csv

============= 346/409
2536_mssql-docker_nondebt.csv

============= 347/409
1519_docker-bash_nondebt.csv

============= 348/409
474_openjdk.csv

============= 349/409
1819_azure-container-networking_nondebt.csv

============

## 分析作業③

今回対象のファイルが分析作業②で取り出したファイル群に含まれているか分析

In [146]:
not_in_col = []
results = os.listdir("./onehot-encoding/")

for result in results:
    
    ## ファイル拡張子が違うものは除外
    if result[-4:] != '.csv':
        continue
        
    df = pd.read_csv(f'./onehot-encoding/{result}', index_col=0)
    id = int(result.split('_')[0])
    target_saikyo = saikyo[ saikyo["id"]==id ]
            
    print(f'\n\n========== {result}')
    print(f'revision : {target_saikyo["revision"].values[0]}')
    print(f'satd-type : {target_saikyo["class"].values[0]}')
    
    project = target_saikyo["project"].values[0]
    print(f'project : {project}')
    target_file = target_saikyo["path"].values[0].replace(project, '').splitlines()
    
    if result == "2009_certbot.csv": ## certbotは .git が頭についていたためエラー
        target_file[0] = target_file[0].replace('.git', '')
         
#     print('targetfile : ', target_file)
    print('-'*10)
    
    
    ## ターゲットファイルがカラム名に含まれているかどうかの確認を行う
    columns = ' '.join(df.columns.tolist()[3:])
    
    FLG = True
    for file in target_file:
        if file[0] == "/":## 検索エラー回避
            file = file[1:]
            
        if file in columns:
            print(True, file)
        else:
            print(False, file)
            FLG = False
            
    if not FLG:
        not_in_col.append(result)
        display(df.head(1))



========== 2141_ghost_nondebt.csv
revision : 6f37d79247ca28f20b3aa4fe2da8f1ba7d527737
satd-type : non-debt
project : docker-library/ghost
----------
True 2/alpine/Dockerfile
True 2/debian/Dockerfile
True 3/alpine/Dockerfile
True 3/debian/Dockerfile


========== 1612_dockerfiles.csv
revision : 44e5012b3b8c47340ad24da5467c7bc3175173f5
satd-type : code/version
project : tianon/dockerfiles
----------
True mojo/Dockerfile


========== 508_php_nondebt.csv
revision : a3cf31c9020559777c731d980047450cba744122
satd-type : non-debt
project : docker-library/php
----------
True 7.2/alpine3.10/cli/Dockerfile
True 7.2/alpine3.10/fpm/Dockerfile
True 7.2/alpine3.10/zts/Dockerfile
True 7.2/alpine3.11/cli/Dockerfile
True 7.2/alpine3.11/fpm/Dockerfile
True 7.2/alpine3.11/zts/Dockerfile
True 7.2/buster/apache/Dockerfile
True 7.2/buster/cli/Dockerfile
True 7.2/buster/fpm/Dockerfile
True 7.2/buster/zts/Dockerfile
True 7.2/stretch/apache/Dockerfile
True 7.2/stretch/cli/Dockerfile
True 7.2/stretch/fpm/Docker

,commitid,date,diff




========== 1588_dockerfiles.csv
revision : 44e5012b3b8c47340ad24da5467c7bc3175173f5
satd-type : test/integrity
project : tianon/dockerfiles
----------
True grafana/Dockerfile


========== 1877_batch-shipyard_nondebt.csv
revision : d6da749f9cd678037bd520bc074e40066ea35b56
satd-type : non-debt
project : Azure/batch-shipyard
----------
True recipes/CNTK-CPU-Infiniband-IntelMPI/docker/Dockerfile
True recipes/CNTK-GPU-Infiniband-IntelMPI/docker/Dockerfile


========== 269_docker.csv
revision : cdd23554f9417055ca2a18805fa9ad5716209934
satd-type : test/integrity
project : docker-library/docker
----------
True 19.03/Dockerfile
True 19.03-rc/Dockerfile


========== 1424_rocker_nondebt.csv
revision : 567713ca260f846dc725fcca10eb58716b9b9532
satd-type : non-debt
project : rocker-org/rocker
----------
True r-apt/cosmic/Dockerfile


========== 1159_docker-nuxeo_nondebt.csv
revision : 321756e6c8d808b1fc225cfdc9287e2bde8ec319
satd-type : non-debt
project : nuxeo/docker-nuxeo
----------
True 10.10/D

,commitid,date,diff




========== 2460_cntk.csv
revision : e9396480025b9ca457d26b6f33dd07c474c6aa04
satd-type : code/workaround
project : microsoft/cntk
----------
True Tools/docker/CNTK-GPU-Image/Dockerfile


========== 472_openjdk_nondebt.csv
revision : bb4265f075850b8b6b36347d0cb045a3fa4df18e
satd-type : non-debt
project : docker-library/openjdk
----------
True 11/jdk/windows/windowsservercore-1809/Dockerfile
True 11/jdk/windows/windowsservercore-ltsc2016/Dockerfile
True 11/jre/windows/windowsservercore-1809/Dockerfile
True 11/jre/windows/windowsservercore-ltsc2016/Dockerfile
True 14/jdk/windows/windowsservercore-1809/Dockerfile
True 14/jdk/windows/windowsservercore-ltsc2016/Dockerfile
True 15/jdk/windows/windowsservercore-1809/Dockerfile
True 15/jdk/windows/windowsservercore-ltsc2016/Dockerfile
True 8/jdk/windows/windowsservercore-1809/Dockerfile
True 8/jdk/windows/windowsservercore-ltsc2016/Dockerfile
True 8/jre/windows/windowsservercore-1809/Dockerfile
True 8/jre/windows/windowsservercore-ltsc2016/Do

...



========== 801_swarm_nondebt.csv
revision : 613bdb260e53c03dc5422719d8b8cc32721e46f9
satd-type : non-debt
project : docker/swarm
----------
True test/integration/Dockerfile


========== 1022_kubernetes-ingress-controller_nondebt.csv
revision : 0aad24a77ed76d1d3245b65b772b22aa33ecb280
satd-type : non-debt
project : Kong/kubernetes-ingress-controller
----------
True Dockerfile


========== 730_wordpress_nondebt.csv
revision : 2f49bc64582ec81d2a821f090993be97b119847a
satd-type : non-debt
project : docker-library/wordpress
----------
True php7.2/cli/Dockerfile
True php7.3/cli/Dockerfile
True php7.4/cli/Dockerfile


========== 638_rabbitmq_nondebt.csv
revision : 29b163923699c18c6d76a95df10ba52665ef8741
satd-type : non-debt
project : docker-library/rabbitmq
----------
True 3.7/alpine/Dockerfile
True 3.7/ubuntu/Dockerfile
True 3.7-rc/alpine/Dockerfile
True 3.7-rc/ubuntu/Dockerfile
True 3.8/alpine/Dockerfile
True 3.8/ubuntu/Dockerfile
False 3.8-rc/alpine/Dockerfile
True 3.8-rc/ubuntu/Dockerf

,commitid,date,diff,3.7.9/ubuntu/Dockerfile,3.8-beta/3.6/alpine/Dockerfile,3.6/debian/Dockerfile-alpine.template,3.7-rc/alpine/Dockerfile,3.8-rc/ubuntu/Dockerfile,3.8/ubuntu/Dockerfile,3.7.9/alpine/Dockerfile,3.7/alpine/Dockerfile,3.7-rc/ubuntu/Dockerfile,3.8/alpine/Dockerfile,3.7/ubuntu/Dockerfile,3.6/debian/Dockerfile-ubuntu.template
0,29b163923699c18c6d76a95df10ba52665ef8741,2020-05-27 10:22:38-07:00,0,0,0,0,0,0,0,0,0,0,0,0,0




========== 1102_docker_nondebt.csv
revision : ba1500b05ee8889c99a46f1d0de63ab40d3e2926
satd-type : non-debt
project : nextcloud/docker
----------
True 16.0/apache/Dockerfile
False 16.0/fpm/Dockerfile
True 17.0/apache/Dockerfile
True 17.0/fpm/Dockerfile
True 18.0/apache/Dockerfile
True 18.0/fpm/Dockerfile
True 19.0-rc/apache/Dockerfile
True 19.0-rc/fpm/Dockerfile


,commitid,date,diff,15.0-rc/fpm/Dockerfile,15.0/apache/Dockerfile,13.0/apache/Dockerfile,18.0/apache/Dockerfile,11.0/apache/Dockerfile,13.0-rc/fpm/Dockerfile,17.0/fpm/Dockerfile,...,19.0-rc/apache/Dockerfile,17.0-rc/fpm/Dockerfile,11.0/fpm/Dockerfile,12.0/fpm/Dockerfile,15.0/fpm/Dockerfile,17.0/apache/Dockerfile,19.0-rc/fpm/Dockerfile,15.0-rc/apache/Dockerfile,18.0/fpm/Dockerfile,15.0-beta/fpm/Dockerfile
0,ba1500b05ee8889c99a46f1d0de63ab40d3e2926,2020-05-23 22:34:08+00:00,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0




========== 2230_jetty.docker_nondebt.csv
revision : be1ee24d7993c7e3e3e6f2978aca3e02ee6f5435
satd-type : non-debt
project : eclipse/jetty.docker
----------
True 10.0-jdk11/Dockerfile
True 10.0-jdk11-slim/Dockerfile
True 10.0-jdk14/Dockerfile
True 10.0-jdk14-slim/Dockerfile
True 10.0-jre11/Dockerfile
True 10.0-jre11-slim/Dockerfile
True 11.0-jdk11/Dockerfile
True 11.0-jdk11-slim/Dockerfile
True 11.0-jdk14/Dockerfile
True 11.0-jdk14-slim/Dockerfile
True 11.0-jre11/Dockerfile
True 11.0-jre11-slim/Dockerfile
True 9.4-jdk11/Dockerfile
True 9.4-jdk11-slim/Dockerfile
True 9.4-jdk14/Dockerfile
True 9.4-jdk14-slim/Dockerfile
True 9.4-jdk8/Dockerfile
True 9.4-jdk8-slim/Dockerfile
True 9.4-jre11/Dockerfile
True 9.4-jre11-slim/Dockerfile
True 9.4-jre8/Dockerfile
True 9.4-jre8-slim/Dockerfile


========== 1514_docker-crux.csv
revision : 78132f1f828e871dab1a541f78b7631ad5d12c11
satd-type : code/baseImage
project : therealprologic/docker-crux
----------
True build/aarch64/Dockerfile


========== 21

In [147]:
### 空白ならば全てのファイルが存在していることになるので大丈夫
not_in_col

['1507_docker-splunk_nondebt.csv',
 '2790_newman_nondebt.csv',
 '944_mq-container_nondebt.csv',
 '940_mq-container_nondebt.csv',
 '934_mq-container_nondebt.csv',
 '626_rabbitmq_nondebt.csv',
 '177_convertigo_nondebt.csv',
 '650_rabbitmq_nondebt.csv',
 '623_rabbitmq.csv',
 '2067_cp-docker-images_nondebt.csv',
 '931_mq-container_nondebt.csv',
 '2404_Azure-Kinect-Sensor-SDK_nondebt.csv',
 '2800_newman_nondebt.csv',
 '651_rabbitmq_nondebt.csv',
 '172_convertigo_nondebt.csv',
 '941_mq-container_nondebt.csv',
 '2064_cp-docker-images_nondebt.csv',
 '2008_certbot_nondebt.csv',
 '175_convertigo_nondebt.csv',
 '951_mq-container_nondebt.csv',
 '3011_docker-selenium_nondebt.csv',
 '3003_docker-selenium_nondebt.csv',
 '942_mq-container_nondebt.csv',
 '3015_docker-selenium_nondebt.csv',
 '1101_docker_nondebt.csv',
 '1492_docker-splunk_nondebt.csv',
 '620_rabbitmq_nondebt.csv',
 '936_mq-container_nondebt.csv',
 '641_rabbitmq_nondebt.csv',
 '638_rabbitmq_nondebt.csv',
 '1102_docker_nondebt.csv']

In [148]:
for l in not_in_col:
    if l[-12:] != "_nondebt.csv":
        print(l)

623_rabbitmq.csv


In [69]:
import subprocess

# for file in not_in_col:
#     cmd = f'rm {file}'
#     cwd = "./onehot-encoding/"
#     subprocess.run(list(cmd.split()), cwd=cwd, encoding='utf-8', stdout=subprocess.PIPE, errors="ignore")

## それぞれのファイルにどれくらいの変更点があったかを確認する

In [149]:
pd.DataFrame.from_dict({"file1": [1, 2, 3, 4], "file2": [2, 3, 4, 5]}).rename(index={0:"追加後 commit 回数", 1:"SATD 追加 commit", 2:"初回コミット日時", 3:"最新リビジョンまでの日数"})

,file1,file2
追加後 commit 回数,1,2
SATD 追加 commit,2,3
初回コミット日時,3,4
最新リビジョンまでの日数,4,5


In [150]:
from datetime import datetime

results = os.listdir("./onehot-encoding/")
outputs = os.listdir("./count/")
error = {"IndexError":[]}

for result in results:

    ## ファイル拡張子が違うものは除外
    if result[-4:] != '.csv':
        continue
#     ## 実行済みのものは ignore
#     if result in outputs:
#         continue
        
    df = pd.read_csv(f'./onehot-encoding/{result}', index_col=0)
    id = int(result.split('_')[0])
    target_saikyo = saikyo[ saikyo["id"]==id ]
            
    print(f'\n\n========== {result}')
    print(f'satd-type : {target_saikyo["class"].values[0]}')
    
    project = target_saikyo["project"].values[0]
    target_file = target_saikyo["path"].values[0].replace(project, '').splitlines()
    print('targetfile : ', target_file)
    print('-'*10)
    
    ## SATDが追加されてからのコミット回数を知りたい
    ## よってSATD追加前の変更コミットを無視する (diff==0, and file==1 )
    for col in df.columns.tolist()[3:]:
        try:
            idx = df[(df["diff"]==1) & (df[col]==1)].index.tolist()[-1]
            ## SATD追加時以前のコミットを０カウント
            df.loc[idx+1:][col] = 0
        except IndexError:
            ## SATD追加ファイルではないらしい
            error["IndexError"].append(f'{result}__{col}')
            df.drop([col], axis=1, inplace=True)
    
    ## 初期コミットの日付を確認
    columnslist = df.columns.tolist()[3:]
    olddates = {}
    elapseddates = {}
    
    for col in columnslist:
        old = datetime.strptime(df[(df[col]==1)&(df["diff"]==1)]["date"].values.tolist()[-1][:-6], '%Y-%m-%d %H:%M:%S')
        now = datetime.strptime(df["date"].values.tolist()[0][:-6], '%Y-%m-%d %H:%M:%S')
        olddates[col] = str(old)
        elapseddates[col] = str(now-old)
        
    ## satd追加日時  
    olddates = pd.Series(olddates)
    ## 経過日数
    elapseddates = pd.Series(elapseddates)
    
    ## 変更があった時のコミット回数
    diff_df = df[df["diff"] == 1].sum()[3:]
    ## 変更がなかった時のコミット回数
    not_diff_df = df[df["diff"] == 0].sum()[3:]
    
    df = pd.DataFrame([not_diff_df, diff_df, olddates, elapseddates]).rename(index={0:"commit 回数", 1:"SATD 追加 commit", 2:"初回コミット日時", 3:"最新リビジョンまでの経過日数"})
    display(df)

    df.to_csv(f'./count/{result}')



========== 2141_ghost_nondebt.csv
satd-type : non-debt
targetfile :  ['/2/alpine/Dockerfile', '/2/debian/Dockerfile', '/3/alpine/Dockerfile', '/3/debian/Dockerfile']
----------


/Users/Kaze/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,2/alpine/Dockerfile,3/alpine/Dockerfile,2/debian/Dockerfile,1/alpine/Dockerfile,3/debian/Dockerfile,1/debian/Dockerfile
commit 回数,85,50,113,18,50,97
SATD 追加 commit,1,1,1,1,1,1
初回コミット日時,2019-01-14 17:59:38,2019-10-22 08:49:15,2018-08-22 07:41:46,2019-01-14 17:59:38,2019-10-22 08:49:15,2017-11-03 10:58:31
最新リビジョンまでの経過日数,"534 days, 2:19:33","253 days, 11:29:56","679 days, 12:37:25","534 days, 2:19:33","253 days, 11:29:56","971 days, 9:20:40"




========== 1612_dockerfiles.csv
satd-type : code/version
targetfile :  ['/mojo/Dockerfile']
----------


,mojo/Dockerfile
commit 回数,0
SATD 追加 commit,1
初回コミット日時,2019-04-25 21:41:43
最新リビジョンまでの経過日数,"398 days, 15:50:22"


...

,Dockerfile,Dockerfile.buildkit
commit 回数,136,1
SATD 追加 commit,2,1
初回コミット日時,2017-09-29 17:09:14,2019-05-22 16:49:55
最新リビジョンまでの経過日数,"971 days, 20:33:28","371 days, 20:52:47"




========== 1594_dockerfiles.csv
satd-type : defect
targetfile :  ['/kafka/Dockerfile']
----------


,kafka/Dockerfile
commit 回数,0
SATD 追加 commit,1
初回コミット日時,2017-11-28 13:43:03
最新リビジョンまでの経過日数,"911 days, 23:49:02"




========== 2837_PowerShell-Docker_nondebt.csv
satd-type : non-debt
targetfile :  ['/release/community-stable/amazonlinux/docker/Dockerfile', '/release/community-stable/oraclelinux/docker/Dockerfile']
----------


,release/community-stable/amazonlinux/docker/Dockerfile,release/community-stable/oraclelinux/docker/Dockerfile
commit 回数,4,5
SATD 追加 commit,1,1
初回コミット日時,2019-10-29 02:47:36,2019-10-23 15:20:07
最新リビジョンまでの経過日数,"239 days, 11:00:45","244 days, 22:28:14"


### ここのエラーなんとかしないといけない

#### SATD追加コメントであるはずなのに diff==1 がないカラム達

In [151]:
error

{'IndexError': ['1304_docker_images_nondebt.csv__ros/eloquent/ubuntu/bionic/ros-core/Dockerfile',
  '1245_docker_images_nondebt.csv__gazebo/11/ubuntu/bionic/gzserver11/Dockerfile',
  '1245_docker_images_nondebt.csv__gazebo/11/ubuntu/bionic/gzweb11/Dockerfile',
  '1245_docker_images_nondebt.csv__gazebo/11/ubuntu/bionic/gzclient11/Dockerfile',
  '1245_docker_images_nondebt.csv__gazebo/11/ubuntu/bionic/libgazebo11/Dockerfile']}

#### 対象ファイルがカラムに存在しない 
SATD変更がきちんと判断できていない

In [152]:
not_in_col

['1507_docker-splunk_nondebt.csv',
 '2790_newman_nondebt.csv',
 '944_mq-container_nondebt.csv',
 '940_mq-container_nondebt.csv',
 '934_mq-container_nondebt.csv',
 '626_rabbitmq_nondebt.csv',
 '177_convertigo_nondebt.csv',
 '650_rabbitmq_nondebt.csv',
 '623_rabbitmq.csv',
 '2067_cp-docker-images_nondebt.csv',
 '931_mq-container_nondebt.csv',
 '2404_Azure-Kinect-Sensor-SDK_nondebt.csv',
 '2800_newman_nondebt.csv',
 '651_rabbitmq_nondebt.csv',
 '172_convertigo_nondebt.csv',
 '941_mq-container_nondebt.csv',
 '2064_cp-docker-images_nondebt.csv',
 '2008_certbot_nondebt.csv',
 '175_convertigo_nondebt.csv',
 '951_mq-container_nondebt.csv',
 '3011_docker-selenium_nondebt.csv',
 '3003_docker-selenium_nondebt.csv',
 '942_mq-container_nondebt.csv',
 '3015_docker-selenium_nondebt.csv',
 '1101_docker_nondebt.csv',
 '1492_docker-splunk_nondebt.csv',
 '620_rabbitmq_nondebt.csv',
 '936_mq-container_nondebt.csv',
 '641_rabbitmq_nondebt.csv',
 '638_rabbitmq_nondebt.csv',
 '1102_docker_nondebt.csv']

### それぞれのSATDで最大どれくらいのコミットがあり、どのファイルなのかを判定する

In [153]:
# ## 各行の最大値の列名を取得する
# col = df.idxmax(axis=1)["commit 回数"]
# print("ファイル名 : ", col)
# print("commit 回数 : ", df.loc["commit 回数", col])